# T1090.003 - Multi-hop Proxy
To disguise the source of malicious traffic, adversaries may chain together multiple proxies. Typically, a defender will be able to identify the last proxy traffic traversed before it enters their network; the defender may or may not be able to identify any previous proxies before the last-hop proxy. This technique makes identifying the original source of the malicious traffic even more difficult by requiring the defender to trace malicious traffic through several proxies to identify its source. A particular variant of this behavior is to use onion routing networks, such as the publicly available TOR network. (Citation: Onion Routing)

In the case of network infrastructure, particularly routers, it is possible for an adversary to leverage multiple compromised devices to create a multi-hop proxy chain within the Wide-Area Network (WAN) of the enterprise.  By leveraging [Patch System Image](https://attack.mitre.org/techniques/T1601/001), adversaries can add custom code to the affected network devices that will implement onion routing between those nodes.  This custom onion routing network will transport the encrypted C2 traffic through the compromised population, allowing adversaries to communicate with any device within the onion routing network.  This method is dependent upon the [Network Boundary Bridging](https://attack.mitre.org/techniques/T1599) method in order to allow the adversaries to cross the protected network boundary of the Internet perimeter and into the organization’s WAN. Protocols such as ICMP may be used as a transport.

## Atomic Tests

### Atomic Test #1 - Psiphon
Psiphon 3 is a circumvention tool from Psiphon Inc. that utilizes VPN, SSH and HTTP Proxy technology to provide you
with uncensored access to Internet.
This process will launch Psiphon 3 and establish a connection. Shortly after it will be shut down via process kill commands.
More information can be found about Psiphon using the following urls
http://s3.amazonaws.com/0ubz-2q11-gi9y/en.html
https://psiphon.ca/faq.html

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: The proxy settings backup file must exist on disk at $env:Temp\proxy-backup.txt

##### Check Prereq Commands:
```powershell
if (Test-Path $env:Temp\proxy-backup.txt) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
if(-not (test-path $env:Temp\proxy-backup.txt)){
$Proxy = (Get-ItemProperty -Path 'HKCU:\Software\Microsoft\Windows\CurrentVersion\Internet Settings' -Name "ProxyServer" -ErrorAction Ignore).ProxyServer
Set-Content $env:Temp\proxy-backup.txt $Proxy}

```
##### Description: The Psiphon executable must exist in the Downloads folder

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\psiphon3.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction ignore -Force | Out-Null
Invoke-WebRequest "https://s3.amazonaws.com/0ubz-2q11-gi9y/psiphon3.exe" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\psiphon3.exe" 

```
##### Description: Batch file containing commands to run must be in src directory 

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1090.003\src\Psiphon.bat") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\T1090.003\src\" -ErrorAction ignore -Force | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1090.003/src/Psiphon.bat" -OutFile "PathToAtomicsFolder\T1090.003\src\Psiphon.bat"

```

In [ ]:
Invoke-AtomicTest T1090.003 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
& "PathToAtomicsFolder\T1090.003\src\Psiphon.bat"
```

In [ ]:
Invoke-AtomicTest T1090.003 -TestNumbers 1

#### Cleanup: 
```powershell
$Proxy = Get-Content $env:Temp\proxy-backup.txt -ErrorAction Ignore
if($null -ne $Proxy) 
{Set-ItemProperty -Path 'HKCU:\Software\Microsoft\Windows\CurrentVersion\Internet Settings' -Name "ProxyServer" -Value $Proxy}
```

In [ ]:
Invoke-AtomicTest T1090.003 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Tor Proxy Usage - Windows
This test is designed to launch the tor proxy service, which is what is utilized in the background by the Tor Browser and other applications with add-ons in order to provide onion routing functionality.
Upon successful execution, the tor proxy will be launched, run for 60 seconds, and then exit. 

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: tor.exe must be installed on the machine 

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\tor\Tor\tor.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://archive.torproject.org/tor-package-archive/torbrowser/11.0.6/tor-win32-0.4.6.9.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\tor.zip"
expand-archive -LiteralPath "PathToAtomicsFolder\..\ExternalPayloads\tor.zip" -DestinationPath "PathToAtomicsFolder\..\ExternalPayloads\tor"

```

In [ ]:
Invoke-AtomicTest T1090.003 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
invoke-expression 'cmd /c start powershell -Command {cmd /c "PathToAtomicsFolder\..\ExternalPayloads\tor\Tor\tor.exe"}'
sleep -s 60
stop-process -name "tor" | out-null
```

In [ ]:
Invoke-AtomicTest T1090.003 -TestNumbers 2

### Atomic Test #3 - Tor Proxy Usage - Debian/Ubuntu/FreeBSD
This test is designed to launch the tor proxy service, which is what is utilized in the background by the Tor Browser and other applications with add-ons in order to provide onion routing functionality.
Upon successful execution, the tor proxy service will be launched. 

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Tor must be installed on the machine 

##### Check Prereq Commands:
```sh
if [ -x "$(command -v tor --version)" ]; then exit 0; else exit 1; fi

```
##### Get Prereq Commands:
```sh
(which apt && sudo apt-get -y install tor) || (which pkg && pkg install -y tor)

```

In [ ]:
Invoke-AtomicTest T1090.003 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
[ "$(uname)" = 'FreeBSD' ] && sysrc tor_enable="YES" && service tor start || sudo systemctl start tor
```

In [ ]:
Invoke-AtomicTest T1090.003 -TestNumbers 3

#### Cleanup: 
```sh
[ "$(uname)" = 'FreeBSD' ] && service tor stop && sysrc -x tor_enable || sudo systemctl stop tor
```

In [ ]:
Invoke-AtomicTest T1090.003 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Tor Proxy Usage - MacOS
This test is designed to launch the tor proxy service, which is what is utilized in the background by the Tor Browser and other applications with add-ons in order to provide onion routing functionality.
Upon successful execution, the tor proxy service will be launched. 

**Supported Platforms:** macos
#### Dependencies:  Run with `sh`!
##### Description: Tor must be installed on the machine 

##### Check Prereq Commands:
```sh
if [ -x "$(command -v tor --version)" ]; then exit 0; else exit 1; fi

```
##### Get Prereq Commands:
```sh
if [ ! -x "$(command -v brew --version)" ]; then /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh keystroke return)"; fi
brew install tor

```

In [ ]:
Invoke-AtomicTest T1090.003 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
osascript -e 'tell application "Terminal" to do script "tor"'
```

In [ ]:
Invoke-AtomicTest T1090.003 -TestNumbers 4

#### Cleanup: 
```sh
killall tor > /dev/null 2>&1
```

In [ ]:
Invoke-AtomicTest T1090.003 -TestNumbers 4 -Cleanup

## Detection
When observing use of Multi-hop proxies, network data from the actual command and control servers could allow correlating incoming and outgoing flows to trace malicious traffic back to its source. Multi-hop proxies can also be detected by alerting on traffic to known anonymity networks (such as [Tor](https://attack.mitre.org/software/S0183)) or known adversary infrastructure that uses this technique.

In context of network devices, monitor traffic for encrypted communications from the Internet that is addressed to border routers.  Compare this traffic with the configuration to determine whether it matches with any configured site-to-site Virtual Private Network (VPN) connections the device was intended to have. Monitor traffic for encrypted communications originating from potentially breached routers that is addressed to other routers within the organization.  Compare the source and destination with the configuration of the device to determine if these channels are an authorized Virtual Private Network (VPN) connections or other encrypted modes of communication. Monitor ICMP traffic from the Internet that is addressed to border routers and is encrypted.  Few if any legitimate use cases exist for sending encrypted data to a network device via ICMP.